In [1]:
from fastapi import FastAPI, HTTPException
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import re
from openai import OpenAI
import anthropic
import os
from fastapi.responses import JSONResponse
import uuid
import requests
import json


In [2]:
OPEN_API_KEY = os.getenv("OPEN_API_KEY")
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
POSTGRES_USER=os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD=os.getenv("POSTGRES_PASSWORD")
POSTGRES_DB=os.getenv("POSTGRES_DB")
POSTGRES_HOST=os.getenv("POSTGRES_HOST")
POSTGRES_PORT=os.getenv("POSTGRES_PORT")

In [3]:
from openai import OpenAI
import numpy as np

openai_client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=OPEN_API_KEY
)

anthropic_client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

## Data Load

In [4]:
# GET COMPELETE PRODUCT DATASET
def get_products():
    """
    Fetches all product details from the external API.

    Args:
        No ARGS

    Returns:
        A dictionary containing product details.
    """
    try:
        response = requests.get(f"http://localhost:8000/products")
        response.raise_for_status()  # Raise an exception for bad status codes
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching product details: {e}")
        return {"error": f"No Product availabel"}

In [5]:
# GET COMPELETE HISTORY OF ORDER BY ORDER ID
def get_orders(order_id: str):
    """
    Fetches order details from the external API.

    Args:
        order_id: The unique identifier of the order.

    Returns:
        A dictionary containing order details and shipment history.
    """
    try:
        response = requests.get(f"http://localhost:8000/orders/{order_id}")
        response.raise_for_status()  # Raise an exception for bad status codes
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching order details: {e}")
        return {"error": f"Failed to fetch shipment details for order ID {order_id}"}
    
async def get_orders_handler(input_data):
    order_id = input_data["order_id"]
    # Fetch and return order details based on the order_id
    return {"order_id": order_id, "details": "Order details go here"}

In [6]:
# GET COMPELETE HISTORY OF SHIPMENT BY SHIPMENT ID
def get_shipment_history(shipment_id: str):
    """
    Fetches shipment details from the external API.

    Args:
        shipment_id: The unique identifier of the shipment.

    Returns:
        A dictionary containing order details and shipment history.
    """
    try:
        response = requests.get(f"http://localhost:8000/shipments/{shipment_id}")
        response.raise_for_status()  # Raise an exception for bad status codes
        return [response.json()]
    except requests.exceptions.RequestException as e:
        print(f"Error fetching order details: {e}")
        return {"error": f"Failed to fetch shipment details for shipment ID {shipment_id}"}
    
async def get_shipment_handler(input_data):
    shipment_id = input_data["shipment_id"]
    # Fetch and return order details based on the order_id
    return {"shipment_id": shipment_id, "details": "Shipment details go here"}

In [7]:
# GET COMPELETE HISTORY OF SHIPMENT BY ORDER ID
def get_order_with_shipment_history(order_id: str):
    """
    Fetches order details from the external API.

    Args:
        order_id: The unique identifier of the order.

    Returns:
        A dictionary containing order details and shipment history.
    """
    try:
        response = requests.get(f"http://localhost:8000/shipment_details/{order_id}")
        response.raise_for_status()  # Raise an exception for bad status codes
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching order details: {e}")
        return {"error": f"Failed to fetch order details for order ID {order_id}"}
    
async def get_shipment_orders_handler(input_data):
    order_id = input_data["order_id"]
    # Fetch and return order details based on the order_id
    return {"order_id": order_id, "details": "Shipment Order details go here"}

In [8]:
knowledge_base = get_products()

In [9]:
tools = [
        {
        "name": "get_orders",
        "description": "This function is used to provide details of a specific order by its unique Order ID. \
                        It includes details like User ID, Product Name, Cost, Quantity of order, Order Date, Shipment ID and current Status Of Order.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The Order ID of the order."
                }
            },
            "required": ["order_id"]
        }
    },
    {
        "name": "get_shipment_history",
        "description": "This function is used to provide complete details of shipment history for a Shipment ID. \
            It helps to track an order from the date it is ordered and different stages of shippment involved ducring package shipments. \
            It also provides each stages of shipment like packed, dispatched, out for delivery, delivered, and returned along with date.",
        "input_schema": {
            "type": "object",
            "properties": {
                "shipment_id": {
                    "type": "string",
                    "description": "The Shipment ID of the shipment."
                }
            },
            "required": ["shipment_id"]
        }
    },
    {
        "name": "get_order_with_shipment_history",
        "description": "This function is used to provide complete details of Order and shipment history for a Provide Order ID.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The Order ID of the order."
                }
            },
            "required": ["order_id"]
        }
    }
]

In [10]:
tool_handlers = {
    "get_orders": get_orders_handler,
    "get_shipment_history": get_shipment_handler,
    "get_order_with_shipment_history": get_shipment_orders_handler,
}


In [11]:
tool_choice={"type": "auto"}

## CHAT BOT

### FUNCTION ONLY

### Function and Knowledge Base

In [12]:
async def anthropic_knowledge_function_call(user_query, knowledge_base, tools):
    prompt = f"""
    You are a support assistant. 
    Your job is to answer user queries based on the following knowledge base and use available tools when required.
    Whenever customers enquire about their order status, ask for Order ID or Shipment ID if it is not available in the conversation.

    Start with the summary of their order in a few lines, and provide details of their order in a new section.

    Knowledge Base:
    {knowledge_base}

    Tools:
    {json.dumps(tools, indent=2)}

    User Query:
    {user_query}

    If the answer is directly available in the knowledge base, respond with it. 
    If you need to use a tool, specify the tool and its parameters.
    Provide answers in readable and formatted text.
    """

    # History and initial messages
    history = []
    messages = [
        {"role": "user", "content": prompt}
    ]

    # Mocked `anthropic_client` for demonstration purposes
    response = anthropic_client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        tools=tools,
        messages=messages
    )

    while response.stop_reason == "tool_use":
        tool_use = next(block for block in response.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        # Dynamically call the appropriate tool handler
        tool_result = await tool_handlers[tool_name](tool_input)

        messages = [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": response.content},
            {
                "role": "user",
                "content": [
                    {
                        "type": "tool_result",
                        "tool_use_id": tool_use.id,
                        "content": str(tool_result),
                    }
                ],
            },
        ]

        response = anthropic_client.messages.create(
            model="claude-3-sonnet-20240229",
            max_tokens=4096,
            tools=tools,
            messages=messages
        )

    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )

    return final_response


In [13]:
user_query = "Provide me complete order shipment details of ORD1001, also provide some usage of the product"
result = await anthropic_knowledge_function_call(user_query, knowledge_base, tools)
print(result)

Summary:
Based on the order details, the product ordered is the Apple iPhone 15 Pro. The iPhone 15 Pro is a premium smartphone that offers top-of-the-line performance, features, and design.

Product Details:
Product Name: Apple iPhone 15 Pro
Description: The latest flagship smartphone from Apple, featuring a powerful A17 Bionic chip, a stunning Super Retina XDR display, and a triple-lens camera system.
Usage: Communication, entertainment, productivity, and more. The iPhone 15 Pro can be used for making calls, browsing the internet, taking photos and videos, accessing productivity apps, and enjoying multimedia content.

Shipment History:
<shipment_history>
Date Ordered: 2023-05-01
Packed: 2023-05-02
Dispatched: 2023-05-03  
Out for Delivery: 2023-05-05
Delivered: 2023-05-06
</shipment_history>

The order ORD1001 for the Apple iPhone 15 Pro was placed on 2023-05-01 and was successfully delivered on 2023-05-06 after going through the stages of packing, dispatch, and out for delivery.


In [14]:
import gradio as gr
import json

# Gradio async wrapper
async def gradio_chatbot(user_query, history):
    # Ensure history is initialized as a list
    history = history or []
    
    # Call your function to get the response
    response = await anthropic_knowledge_function_call(user_query, knowledge_base, tools)
    
    # Append the new query and response to history
    history.append((user_query, response))
    
    # Return the updated history for chatbot UI and the same history as state
    return history, history

# Gradio interface setup
with gr.Blocks() as demo:
    chatbot_ui = gr.Chatbot(label="AI Chatbot with Tools")
    message = gr.Textbox(label="Your Query", placeholder="Ask me anything...", lines=1)
    state = gr.State()  # For chat history
    send_button = gr.Button("Send")

    send_button.click(
        fn=gradio_chatbot,
        inputs=[message, state],
        outputs=[chatbot_ui, state],
        api_name="chat"
    )

# Launch the app
demo.launch(server_port=7862)


/opt/anaconda3/envs/trainenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/trainenv/lib/python3.12/site-packages/gradio/components/chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
